In [1]:
from __future__ import print_function
import os
import sys
import time
import numpy as np
import re
import faiss
from multiprocessing.dummy import Pool as ThreadPool
from matplotlib import pyplot

In [37]:
def mmap_fvecs(fname):
    x = np.memmap(fname, dtype='int32', mode='r')
    d = x[0]
    return x.view('float32').reshape(-1, d + 1)[:, 1:]


def mmap_bvecs(fname):
    x = np.memmap(fname, dtype='uint8', mode='r')
    d = x[:4].view('int32')[0]
    return x.reshape(-1, d + 4)[:, 4:]

def ivecs_read(fname):
    a = np.fromfile(fname, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy()

dbname        = 'SIFT100M'
index_key     = 'OPQ16,IVF4096,PQ16'


tmpdir = '../trained_CPU_indexes/bench_cpu_{}_{}'.format(dbname, index_key)

if not os.path.isdir(tmpdir):
    raise("%s does not exist")


#################################################################
# Prepare dataset
#################################################################


print("Preparing dataset", dbname)

if dbname.startswith('SIFT'):
    # SIFT1M to SIFT1000M
    dbsize = int(dbname[4:-1])
    xb = mmap_bvecs('../bigann/bigann_base.bvecs')
    xq = mmap_bvecs('../bigann/bigann_query.bvecs')
    xt = mmap_bvecs('../bigann/bigann_learn.bvecs')

    # trim xb to correct size
    xb = xb[:dbsize * 1000 * 1000]

    gt = ivecs_read('../bigann/gnd/idx_%dM.ivecs' % dbsize)

elif dbname == 'Deep1B':
    xb = mmap_fvecs('../deep1b/base.fvecs')
    xq = mmap_fvecs('../deep1b/deep1B_queries.fvecs')
    xt = mmap_fvecs('../deep1b/learn.fvecs')
    # deep1B's train is is outrageously big
    xt = xt[:10 * 1000 * 1000]
    gt = ivecs_read('../deep1b/deep1B_groundtruth.ivecs')

else:
    print('unknown dataset', dbname, file=sys.stderr)
    sys.exit(1)


print("sizes: B %s Q %s T %s gt %s" % (
    xb.shape, xq.shape, xt.shape, gt.shape))

nq, d = xq.shape
nb, d = xb.shape
assert gt.shape[0] == nq


#################################################################
# Load Index
#################################################################

def get_populated_index():

    filename = "%s/%s_%s_populated.index" % (
        tmpdir, dbname, index_key)

    if not os.path.exists(filename):
        raise("Index does not exist!")
    else:
        print("loading", filename)
        index = faiss.read_index(filename)
    return index


index = get_populated_index()

Preparing dataset SIFT100M
sizes: B (100000000, 128) Q (10000, 128) T (100000000, 128) gt (10000, 1000)
loading ../trained_CPU_indexes/bench_cpu_SIFT100M_OPQ16,IVF4096,PQ16/SIFT100M_OPQ16,IVF4096,PQ16_populated.index


In [47]:
HBM_bank_num = int(9) 
cluster_num   = 4096
PQ_bytes      = 16

folder_name = 'FPGA_data_{}_{}_HBM_{}_banks'.format(dbname, index_key, HBM_bank_num)


if not os.path.exists('/home/wejiang/saved_npy_data/' + folder_name):
    os.mkdir('/home/wejiang/saved_npy_data/' + folder_name)

In [48]:
""" Part 1: Save index, quantizer, and HBM related contents """

' Part 1: Save index, quantizer, and HBM related contents '

In [49]:
""" Get OPQ Matrix """
linear_trans = faiss.downcast_VectorTransform(index.chain.at(0))
OPQ_mat = faiss.vector_to_array(linear_trans.A)
OPQ_mat = OPQ_mat.reshape((128,128))
OPQ_mat = np.array(OPQ_mat, dtype=np.float32)
print(OPQ_mat, OPQ_mat.shape)

[[-0.05746975  0.13880756  0.03853012 ...  0.16249856  0.02070794
  -0.15795065]
 [-0.0051081  -0.01647936 -0.07434352 ...  0.04703823  0.01883231
  -0.1624942 ]
 [ 0.10481689 -0.00398999 -0.09858719 ... -0.00828476  0.11390419
  -0.04354695]
 ...
 [ 0.12617084  0.09430577 -0.08268221 ... -0.06733439  0.04732419
   0.03877737]
 [-0.12709847  0.10950942 -0.05156725 ... -0.07881732  0.05719862
   0.0918511 ]
 [-0.19426978  0.01307625 -0.03762694 ...  0.01201456 -0.05358534
  -0.02917147]] (128, 128)


In [50]:
""" Get IVF index (coarse quantizer) and product quantizer"""

downcasted_index = faiss.downcast_index(index.index)

def get_sub_quantizer_centroids(index):
    """
    return the sub-quantizer centroids, 
    shape = (m, 256, d / m)
    e.g., d=128, m=16 -> (16, 256, 8)
    """
    pq = index.pq
    cen = faiss.vector_to_array(pq.centroids)
    cen = cen.reshape(pq.M, pq.ksub, pq.dsub)
    
    return cen

def get_coarse_quantizer_centroids(index):
    """
    return the coarse-grained quantizer centroids,
    shape = (nlist, d),
    e.g., nlist=1024, d=128 -> (1024, 128)
    """
    coarse_quantizer = faiss.downcast_index(index.quantizer)
    coarse_cen = faiss.vector_to_array(coarse_quantizer.xb)

    coarse_cen = coarse_cen.reshape(coarse_quantizer.ntotal, coarse_quantizer.d)
    return coarse_cen

# Get Sub quantizer info
sub_cen = get_sub_quantizer_centroids(downcasted_index)
print("==== Sub-quantizer ====\n{}\n\nshape:{}\n".format(sub_cen, sub_cen.shape))

# Get Coarse quantizer info
coarse_cen = get_coarse_quantizer_centroids(downcasted_index)
print("==== Coarse-quantizer ====\n{}\n\nshape:{}\n".format(coarse_cen, coarse_cen.shape))

==== Sub-quantizer ====
[[[ 23.96711     26.0044     -11.1172905  ...  11.804664    13.445307
   -50.044632  ]
  [ 16.160625    42.081387   -21.540838   ...  30.668299    18.182981
   -46.038185  ]
  [ -3.708279    14.769348   -12.837059   ...  23.007517    21.180435
    -9.63418   ]
  ...
  [  3.9408724   -2.4239244   -4.9150043  ...  -3.6745992  -17.772942
     7.616997  ]
  [ -3.3290033  -14.343654    -5.694798   ... -19.215685    13.159717
    10.457293  ]
  [ -3.066225     7.14629      7.102779   ...  19.87864    -13.899342
    17.691605  ]]

 [[ -0.09630945  -6.247671    -9.651321   ... -30.396086   -14.633472
    -8.909883  ]
  [ -3.3527985   16.259548   -11.938261   ... -43.37426     -6.179365
     0.73788345]
  [  0.49349424 -13.503662   -21.51919    ... -19.631287    -2.4266634
   -29.517004  ]
  ...
  [ -6.578046     9.059433    40.357506   ...  18.38811    -17.480762
    38.430927  ]
  [-12.9572935   -4.3899236   34.1133     ...  37.677284     4.1169815
     6.0538154 ]
  [

In [51]:
# Save the coarse quantizer and the product quantizer

OPQ_mat = np.array(OPQ_mat, dtype=np.float32)
PQ_quantizer = np.array(sub_cen, dtype=np.float32)
coarse_cen = np.array(coarse_cen, dtype=np.float32)

# 16, 256, 8 -> (0,0,0:8) the first row of the subquantizer of the first sub-vector 
print(OPQ_mat.shape, PQ_quantizer.shape, coarse_cen.shape)

OPQ_mat.tofile("/home/wejiang/saved_npy_data/{}/OPQ_matrix_float32_{}_{}_raw".format(
    folder_name, OPQ_mat.shape[0], OPQ_mat.shape[1]))
PQ_quantizer.tofile("/home/wejiang/saved_npy_data/{}/product_quantizer_float32_{}_{}_{}_raw".format(
    folder_name, PQ_quantizer.shape[0], PQ_quantizer.shape[1], PQ_quantizer.shape[2]))
coarse_cen.tofile("/home/wejiang/saved_npy_data/{}/vector_quantizer_float32_{}_{}_raw".format(
    folder_name, coarse_cen.shape[0], coarse_cen.shape[1]))

(128, 128) (16, 256, 8) (4096, 128)


In [52]:
""" Get contents in a single Voronoi Cell """
invlists = downcasted_index.invlists

def get_invlist(invlists, l):
    """ 
    returns the (vector IDs set, PQ cose set) of list ID "l"
    list_ids: (#vec_in_list, ), e.g., #vec_in_list=10 -> (10, )
    list_codes: (#vec_in_list, m), e.g., #vec_in_list=10, m=16 -> (10, 16)
    
    That the data is *NOT* copied: if the inverted index is deallocated or changes, accessing the array may crash.
    To avoid this, just clone the output arrays on output. 
    """
    ls = invlists.list_size(l)
    list_vec_ids = faiss.rev_swig_ptr(invlists.get_ids(l), ls)
    list_PQ_codes = faiss.rev_swig_ptr(invlists.get_codes(l), ls * invlists.code_size)
    list_PQ_codes = list_PQ_codes.reshape(-1, invlists.code_size)
    return list_vec_ids, list_PQ_codes


In [53]:
# Example of using function "get_invlist"
list_id = 123
list_vec_ids, list_PQ_codes = get_invlist(invlists, list_id)
print("==== Vector IDs ====\n{}\n\nshape: {}\n".format(list_vec_ids, list_vec_ids.shape))
print("==== PQ codes ====\n{}\n\nshape: {}\n".format(list_PQ_codes, list_PQ_codes.shape))

==== Vector IDs ====
[    4509     8232    13925 ... 99993735 99995854 99998517]

shape: (30435,)

==== PQ codes ====
[[ 10 160  30 ...  94 175 155]
 [227  29 155 ...  74  23  20]
 [234 176  99 ... 134 191 183]
 ...
 [110 152 217 ... 123 147  54]
 [243 142 185 ...  37 215  50]
 [154 236 196 ... 207 246  75]]

shape: (30435, 16)



In [54]:
def get_contents_to_HBM(invlists, cluster_id, HBM_bank_num=int(21)):
    """
    For a single cluster (list), extract the contents in the format that HBM loads
      inputs:
        invlists: the Faiss index.invlists object
        cluster_id: e.g., 0~8191 for nlist=8192
        HBM_bank_num: 21 for default, athough there are 32 banks on U280, 
                    we don't have enough hardware logic to load and compute at that rate
      outputs:
        HBM_bank_contents( content of 21 banks): a list of 21 element
            each element is a byte object with a set of contents
            the size of the content is m * 64 bytes
            the contents includes (3 * (int32 vector ID) (16 byte PQ code)) + 4byte padding
        entries_per_bank: int, all HBM shares the same number of 512-bit items to scan
        last_valid_element: int from 0 to 62 (63 numbers in total given 21 HBM channels)
            some of the elements in the last row are paddings, which of them is the last non-padding (valid) 
            
      term:
        entry: a 512-bit entry containing 3 PQ codes
        vector: a 20-byte vector containing 4 byte vector ID + 16 byte PQ code
    """
    
    list_vec_ids, list_PQ_codes = get_invlist(invlists, cluster_id)
#     print("list_vec_ids", list_vec_ids.shape)
#     print("list_PQ_codes", list_PQ_codes.shape)
    num_vec = list_vec_ids.shape[0]
    assert list_vec_ids.shape[0] == list_PQ_codes.shape[0]
    
#     print("num_vec", num_vec)
    
    if num_vec % (HBM_bank_num * 3) == 0:
        # no padding
        entries_per_bank = num_vec / (HBM_bank_num * 3)
        last_valid_element = HBM_bank_num * 3 - 1
        num_vec_per_HBM = [int(num_vec / HBM_bank_num)] * HBM_bank_num
        num_pad_per_HBM = [0] * HBM_bank_num
    else:
        # with padding
        entries_per_bank = int(num_vec / (HBM_bank_num * 3)) + 1
        last_valid_element = num_vec % (HBM_bank_num * 3) - 1
        num_vec_per_HBM = []
        num_pad_per_HBM = []
        
        counted_banks = 0
        # bank with full valid elements
        for i in range(int((last_valid_element + 1) / 3)):
            num_vec_per_HBM += [entries_per_bank * 3]
            num_pad_per_HBM += [0]
        counted_banks += int((last_valid_element + 1) / 3)
        
        # (optional) bank with some valid elements and some padding in the last entry
        if (last_valid_element + 1) % 3 != 0:
            num_vec_per_HBM += [(entries_per_bank - 1) * 3 + (last_valid_element + 1) % 3]
            num_pad_per_HBM += [3 - (last_valid_element + 1) % 3]
            counted_banks += 1
        
        # (optional) bank with full padding in the last entry
        for i in range(HBM_bank_num - counted_banks):
            num_vec_per_HBM += [int((entries_per_bank - 1) * 3)]
            num_pad_per_HBM += [3]
            
    assert np.sum(np.array(num_vec_per_HBM)) == num_vec
    assert entries_per_bank * HBM_bank_num * 3 - np.sum(np.array(num_pad_per_HBM)) == num_vec
    
    HBM_bank_contents = []
    
    start = int(0)
    
    zero = int(0)
    empty_byte = zero.to_bytes(1, "little", signed=True)
    
#     print("num_vec_per_HBM:", num_vec_per_HBM)
#     print("num_pad_per_HBM:", num_pad_per_HBM)
    
    for i in range(HBM_bank_num):
        
        # add valid vectors first
        end = start + num_vec_per_HBM[i]
        vec_per_bank_count = 0
        byte_obj = bytes()
        
#         print(start, end)
        
        for vec_id_per_bank in range(start, end):
            
            # Vec ID = signed int
            vec_id = int(list_vec_ids[vec_id_per_bank])
            # Xilinx's ap int use little endian
            # Linux on X86 use little endian
            # https://serverfault.com/questions/163487/how-to-tell-if-a-linux-system-is-big-endian-or-little-endian
            byte_obj += vec_id.to_bytes(4, "little", signed=True)
            
            # PQ code = unsigned char
            PQ_codes = list_PQ_codes[vec_id_per_bank]
            for code in PQ_codes:
                code = int(code)
                # Xilinx's ap int use little endian
                byte_obj += code.to_bytes(1, "little", signed=False)
            
            vec_per_bank_count += 1
            if vec_per_bank_count % 3 == 0:
                byte_obj += empty_byte * 4
        
        start = end
        
        # then add paddings
        if num_pad_per_HBM[i] > 0:
            for pad_id in range(num_pad_per_HBM[i]):
                byte_obj += empty_byte * 20
            byte_obj += empty_byte * 4
        
        HBM_bank_contents += [byte_obj]
       
    for i in range(HBM_bank_num):
        assert len(HBM_bank_contents[i]) == len(HBM_bank_contents[0])
        assert len(HBM_bank_contents[i]) == 64 * entries_per_bank
    
    return HBM_bank_contents, entries_per_bank, last_valid_element

In [55]:
# Get HBM contents from all clusters

list_HBM_bank_contents = [] # array of cluster_num * HBM_bank_num elements
list_entries_per_bank = []
list_last_valid_element = []

for c in range(cluster_num):
    HBM_bank_contents, entries_per_bank, last_valid_element = get_contents_to_HBM(invlists, c, HBM_bank_num)
    list_HBM_bank_contents += HBM_bank_contents
    list_entries_per_bank += [entries_per_bank]
    list_last_valid_element += [last_valid_element]

In [56]:
# Reorder list_HBM_bank_contents

print(len(list_HBM_bank_contents))
print("list_entries_per_bank:\n", list_entries_per_bank)
print("list_last_valid_element:\n", list_last_valid_element)

list_HBM_bank_contents_reordered = [] # put all contents of the same HBM bank together

for b in range(HBM_bank_num):
    sub_list = []
    for c in range(cluster_num):
        sub_list += [list_HBM_bank_contents[c * HBM_bank_num + b]]
    print(len(sub_list), len(sub_list[0]))
    list_HBM_bank_contents_reordered += [sub_list]
    
print("list_HBM_bank_contents_reordered:", len(list_HBM_bank_contents_reordered), len(list_HBM_bank_contents_reordered[0]))

36864
list_entries_per_bank:
 [827, 840, 722, 818, 647, 786, 1180, 940, 618, 1066, 1110, 1006, 819, 1132, 938, 1169, 928.0, 463, 673, 808, 921, 776, 1251, 73, 698, 750.0, 1009, 668, 683.0, 934, 1356, 1201, 1288, 804, 1071.0, 1710, 720, 1332, 974, 1915, 968, 927, 1858, 984, 876, 805, 796, 819, 1774, 851, 892, 1874, 805, 962, 992, 705, 469, 1238, 562, 692, 774, 549, 764, 815, 198, 822, 388, 1851, 588, 645, 796, 612, 606, 708, 1285, 658, 368, 1200, 641, 821, 701.0, 851, 906, 939, 921, 1026, 779, 1079, 1587, 1034, 703, 858, 1576, 736, 612, 580, 811, 1035, 937, 863, 663, 710, 816, 1869, 990, 742, 676, 516, 1005, 546, 1178, 831, 2054, 760.0, 914, 1634, 588, 1129, 838, 752, 606, 883, 1393, 1128, 964, 1250, 1755, 339, 868, 1091, 1222, 801, 700, 990, 694, 873, 765, 777, 814, 806.0, 704, 683, 1229, 525, 1578.0, 1063, 1061, 1262, 106.0, 745, 995, 1028, 761, 1189.0, 863, 815, 2639, 1101, 685, 1002, 1063, 798, 986, 431, 503, 815, 846, 1061, 872, 854, 680, 893, 804, 1023, 923.0, 608, 1288, 734, 765,

In [57]:
# Concatenate 

HBM_bank_contents_all = [bytes()] * HBM_bank_num # contents of each bank
for b in range(HBM_bank_num):
    HBM_bank_contents_all[b] = HBM_bank_contents_all[b].join(list_HBM_bank_contents_reordered[b])
    
total_size = np.sum(np.array([len(h) for h in HBM_bank_contents_all]))
print("HBM_bank_contents_all: shape: {}\tsize: {}".format(len(HBM_bank_contents_all), total_size))

HBM_bank_contents_all: shape: 9	size: 2134477440


In [58]:
# Save HBM contents 

for b in range(HBM_bank_num):
    assert len(HBM_bank_contents_all[b]) == len(HBM_bank_contents_all[0])

for b in range(HBM_bank_num):
    with open ('/home/wejiang/saved_npy_data/{}/HBM_bank_{}_raw'.format(folder_name, b), 'wb') as f:
        f.write(HBM_bank_contents_all[b])

In [59]:
# Save control contents

#  The format of storing HBM_info_start_addr_and_scanned_entries_every_cell_and_last_element_valid: 
#     8192 start_addr, then 8192 scanned_entries_every_cell, then 8192 last_valid_element
#     int start_addr_LUT[nlist];
#     int scanned_entries_every_cell_LUT[nlist];
#     int last_valid_channel_LUT[nlist];  

list_start_addr_every_cell = [0]
for c in range(cluster_num - 1):
    list_start_addr_every_cell.append(list_start_addr_every_cell[c] + list_entries_per_bank[c])

assert len(list_start_addr_every_cell) == len(list_entries_per_bank) and\
    len(list_start_addr_every_cell) == len(list_last_valid_element)

print(list_start_addr_every_cell[-1])

HBM_info_start_addr_and_scanned_entries_every_cell_and_last_element_valid = \
    list_start_addr_every_cell + list_entries_per_bank + list_last_valid_element

HBM_info_start_addr_and_scanned_entries_every_cell_and_last_element_valid = np.array(
    HBM_info_start_addr_and_scanned_entries_every_cell_and_last_element_valid, dtype=np.int32)

HBM_info_start_addr_and_scanned_entries_every_cell_and_last_element_valid.tofile(
    "/home/wejiang/saved_npy_data/{}/HBM_info_start_addr_and_scanned_entries_every_cell_and_last_element_valid_3_by_{}_raw".format(
        folder_name, cluster_num))

3704998.0


In [60]:
""" Part 2: Save results of different nprobe """

' Part 2: Save results of different nprobe '

In [61]:
#################################################################
# Perform searches and save results
#################################################################


parametersets = ['nprobe=17', 'nprobe=18', 'nprobe=32']

topK=10

ps = faiss.ParameterSpace()
ps.initialize(index)

# make sure queries are in RAM
xq = xq.astype('float32').copy()
xq = np.array(xq, dtype=np.float32)
xq.tofile("/home/wejiang/saved_npy_data/{}/query_vectors_float32_{}_{}_raw".format(folder_name, xq.shape[0], xq.shape[1]))

# a static C++ object that collects statistics about searches
ivfpq_stats = faiss.cvar.indexIVFPQ_stats
ivf_stats = faiss.cvar.indexIVF_stats


# we do queries in a single thread
faiss.omp_set_num_threads(1)

print(' ' * len(parametersets[0]), '\t', 'R@10   time    %pass')
# print(' ' * len(parametersets[0]), '\t', 'R@1    R@10   R@100     time    %pass')

for param in parametersets:
    print(param, '\t', end=' ')
    sys.stdout.flush()
    ps.set_index_parameters(index, param)
    t0 = time.time()
    ivfpq_stats.reset()
    ivf_stats.reset()
    faiss_result_dist, faiss_result_idx = index.search(xq, topK)
    t1 = time.time()
    for rank in [topK]:
        n_ok = (faiss_result_idx[:, :rank] == gt[:, :1]).sum()
        print("%.4f" % (n_ok / float(nq)), end=' ')
    print("%8.3f  " % ((t1 - t0) * 1000.0 / nq), end=' ')
    print("%5.2f" % (ivfpq_stats.n_hamming_pass * 100.0 / ivf_stats.ndis))
    
    D = np.array(faiss_result_dist, dtype=np.float32)
    I = np.array(faiss_result_idx, dtype=np.int32)
    print(D.shape, I.shape, xq.shape)
    D.tofile("/home/wejiang/saved_npy_data/{}/result_nprobe_{}_distance_float32_{}_{}_raw".format(
        folder_name, param[-2:], D.shape[0], D.shape[1]))
    I.tofile("/home/wejiang/saved_npy_data/{}/result_nprobe_{}_index_int32_{}_{}_raw".format(
        folder_name, param[-2:], I.shape[0], I.shape[1]))

          	 R@10   time    %pass
nprobe=17 	 0.8015    5.582    0.00
(10000, 10) (10000, 10) (10000, 128)
nprobe=18 	 0.8031    5.904    0.00
(10000, 10) (10000, 10) (10000, 128)
nprobe=32 	 0.8191   10.269    0.00
(10000, 10) (10000, 10) (10000, 128)
